# ***TUGAS KELOMPOK METODE NUMERIK EIGEN VALUE APPROXIMATION***
Anggota:
1. Shofwan Fikrul Huda(24060124130106)

Kelas: E
___

#### **A. Definisi**

Nilai Eigen adalah bagian skalar khusus yang memiliki hubungan dengan sistem persamaan linear. Hal ini banyak digunakan dalam persamaan matriks. Eigen value memiliki rumus dasarsebagai berikut:

$Ax = λx$

Angka atau nilai skalar “λ” adalah nilai eigen dari A.

#### **B. Teknik aproksimasi eigen value**

Dalam algoritma eigen value memliki beberapa pendekatan yang dapat dilakukan. di bagian ini, kita akan membahas 4 teknik pendekatan yang dapat dilakukan.

In [ ]:
import numpy as np

**1. Power iteration**



Metode iterasi pangkat adalah sebuah algoritma untuk menemukan **nilai eigen dominan** (nilai eigen dengan magnitudo terbesar) dan **vektor eigen** yang bersesuaian dari sebuah matriks. Metode ini bekerja dengan mengalikan matriks secara berulang dengan sebuah vektor tebakan awal, yang akan konvergen ke arah vektor eigen dominan.

### Rumus Utama

1.  **Inisialisasi**:

    Pilih sebuah vektor tebakan awal $v_0$ yang tidak nol. Biasanya, $v_0$ dipilih sebagai vektor dengan semua elemennya bernilai 1.

  $v_0$ = \begin{pmatrix} 1 \\ 1 \\ \vdots \\ 1 \end{pmatrix}

2.  **Iterasi**:
    Untuk $k = 0, 1, 2, \dots$ hingga konvergensi:
    * Hitung $w_{k+1} = A v_k$
        $$w_{k+1} = A v_k$$
    * Normalisasi $v_{k+1} = \frac{w_{k+1}}{\|w_{k+1}\|}$
        $$v_{k+1} = \frac{w_{k+1}}{\|w_{k+1}\|_2}$$
        di mana $\| \cdot \|_2$ adalah norma Euclidean (L2-norm).

3.  **Konvergensi**:
    Iterasi dihentikan jika perubahan antara $v_{k+1}$ dan $v_k$ cukup kecil, yaitu:
    $$\|v_{k+1} - v_k\|_2 < \text{toleransi}$$

4.  **Estimasi Nilai Eigen (Rayleigh Quotient)**:
    Setelah vektor eigen $v$ (hasil konvergensi $v_k$) ditemukan, nilai eigen dominan $\lambda$ dapat diestimasi menggunakan **Rasio Rayleigh**:
    $$\lambda = \frac{v^T A v}{v^T v}$$
    Karena $v$ adalah vektor ternormalisasi ($v^T v = \|v\|_2^2 = 1$), rumusnya dapat disederhanakan menjadi:
    $$\lambda = v^T A v$$

In [ ]:
def power_iteration(A, max_iterations=1000, tolerance=1e-6):
    # Membuat matriks tebakan awal
    v = np.array(np.ones(A.shape[0]), dtype=float)
    for i in range(max_iterations):
        wi = A @ v
        vi = wi / np.linalg.norm(wi)
        if np.linalg.norm(vi - v) < tolerance:
            break
        v = vi
    # Menghitung eigenvalue menggunakan ray leigh rasio
    eigenvalue = (v.T @ A @ v)/(v.T @ v)
    return eigenvalue


In [ ]:
A = np.array([
    [1, 2, 3, 1, 5],
    [2, 3, 4, 5, 6],
    [3, 4, 5, 3, 7],
    [1, 5, 3, 4, 8],
    [5, 6, 7, 8, 9]
], dtype=float)
eigenvalues = power_iteration(A)
print("Matrix A:\n", A)
print("Eigenvalues of A:\n", eigenvalues)

Matrix A:
 [[1. 2. 3. 1. 5.]
 [2. 3. 4. 5. 6.]
 [3. 4. 5. 3. 7.]
 [1. 5. 3. 4. 8.]
 [5. 6. 7. 8. 9.]]
Eigenvalues of A:
 24.249542966441247


**2. Reduksi Householder**



Metode reduksi Householder digunakan untuk mentransformasi sebuah matriks $A$ menjadi bentuk yang lebih sederhana (biasanya **tridiagonal** jika $A$ simetris, atau **Hessenberg atas** jika $A$ tidak simetris) melalui serangkaian refleksi. Transformasi ini adalah transformasi similaritas, sehingga **nilai eigen** dari matriks asli tetap terjaga.

### Rumus Utama dan Langkah-Langkah

Untuk setiap langkah ke-$k$ (dimana $k$ berjalan dari kolom pertama hingga kolom ke-$n-2$):

1.  **Pilih Sub-vektor**:
    Ambil vektor kolom $x$ dari matriks $A$ saat ini, yaitu $A_{k+1:n, k}$. Ini adalah vektor yang elemen-elemennya (kecuali yang pertama, relatif terhadap $x$ itu sendiri) ingin kita nolkan.

$$
x = \begin{pmatrix} A_{k+1,k} \\ A_{k+2,k} \\ \vdots \\ A_{n,k} \end{pmatrix}
$$
    

2.  **Bentuk Vektor Householder $v$**:
    Vektor $v$ mendefinisikan hyperplane refleksi.
    $$
    v = x + \text{sign}(x_1) \|x\|_2 e_1
    $$
    di mana $x_1$ adalah elemen pertama dari vektor $x$, dan $e_1 = (1, 0, \dots, 0)^T$ adalah vektor basis standar dengan dimensi yang sama seperti $x$. Vektor $v$ kemudian dinormalisasi:
    $$
    v = \frac{v}{\|v\|_2}
    $$

3.  **Bentuk Matriks Reflektor Householder $H_{sub}$**:
    Matriks reflektor $H_{sub}$ yang akan diterapkan pada sub-blok matriks $A$ adalah:
    $$
    H_{sub} = I - 2vv^T
    $$
    di mana $I$ adalah matriks identitas dengan ukuran yang sesuai.

4.  **Bentuk Matriks Reflektor Penuh $H_k$**:
  Matriks $H_{sub}$ disematkan ke dalam matriks identitas berukuran $n \times n$ untuk membentuk $H_k$:
  $$
  H_k = \begin{pmatrix} I_k & 0 \\ 0 & H_{sub} \end{pmatrix}
  $$
  di mana $I_k$ adalah matriks identitas berukuran $(k+1) \times (k+1)$

5.  **Terapkan Transformasi Similaritas**:
    Matriks $A$ diperbarui sebagai berikut:
    $$
    A_{\text{baru}} = H_k A H_k^T
    $$
    Karena $H_k$ adalah matriks ortogonal dan simetris ($H_k = H_k^T$), ini bisa ditulis juga sebagai:
    $$
    A_{\text{baru}} = H_k A H_k
    $$
    Transformasi ini mengenolkan elemen-elemen yang diinginkan di bawah sub-diagonal kolom ke-$k$ dan, jika $A$ simetris, juga elemen-elemen yang bersesuaian di baris ke-$k$.

---
Proses ini diulang untuk $k = 0, 1, \dots, n-3$. Hasil akhirnya adalah matriks tridiagonal (jika $A$ simetris).

In [ ]:
def householder(A):
    A = A.copy().astype(float)      # Salin matriks, pastikan tipe float.
    n = A.shape[0]                  # Ukuran matriks.
    for k in range(n - 2):
        x = A[k+1:, k]              # Sub-vektor kolom di bawah diagonal utama yang akan di-nol-kan.
        e1 = np.zeros(x.shape[0])
        e1[0] = 1.0
        v_sign = np.sign(x[0]) if x[0] != 0 else 1.0
        v = x + v_sign * np.linalg.norm(x) * e1
        if np.linalg.norm(v) > 1e-15: # Hindari pembagian dengan nol
            v = v / np.linalg.norm(v) # Normalisasi v.

        # H_sub: Matriks reflektor Householder kecil (I - 2*v*v^T).
        H_sub = np.eye(n-k-1) - 2.0 * (v[:, None] @ v[None, :])

        Hk = np.eye(n)
        Hk[k+1:, k+1:] = H_sub      # Sematkan H_sub ke Hk.

        # Terapkan transformasi similaritas: A_baru = Hk @ A @ Hk.
        A = Hk @ A @ Hk
    return A




In [ ]:


# Matriks input
A = np.array([
    [1, 2, 3, 1, 5],
    [2, 3, 4, 5, 6],
    [3, 4, 5, 3, 7],
    [1, 5, 3, 4, 8],
    [5, 6, 7, 8, 9]
], dtype=float)
print("Hasil matriks A setelah reduksi Householder:")
print(np.round(householder(A), 4))

Hasil matriks A setelah reduksi Householder:
[[ 1.     -6.245   0.     -0.     -0.    ]
 [-6.245  20.0513 -7.5361 -0.      0.    ]
 [ 0.     -7.5361  1.5273  2.1574  0.    ]
 [-0.     -0.      2.1574  0.0951  0.793 ]
 [-0.      0.      0.      0.793  -0.6736]]


**3. QR method**

Metode QR adalah algoritma iteratif yang digunakan untuk menghitung **nilai eigen** dan vektor eigen dari sebuah matriks. Versi dasar dari metode ini bekerja dengan melakukan dekomposisi QR secara berulang dan mengalikan faktor-faktornya dalam urutan terbalik, yang menyebabkan matriks konvergen ke bentuk di mana nilai eigen mudah dibaca dari diagonalnya.

### Langkah-Langkah dan Rumus Utama

Misalkan $A_0 = A$ adalah matriks awal.

1.  **Iterasi**:
    Untuk $k = 0, 1, 2, \dots$ hingga konvergensi atau jumlah iterasi maksimum tercapai:
    * **Dekomposisi QR**: Faktorkan matriks $A_k$ menjadi produk dari matriks ortogonal $Q_k$ dan matriks segitiga atas $R_k$.
        $$
        A_k = Q_k R_k
        $$
    * **Bentuk Matriks Berikutnya**: Kalikan $R_k$ dan $Q_k$ dalam urutan terbalik untuk mendapatkan matriks $A_{k+1}$.
        $$
        A_{k+1} = R_k Q_k
        $$
        Perhatikan bahwa $A_{k+1} = R_k Q_k = Q_k^T A_k Q_k$ (karena $Q_k$ ortogonal, $Q_k^{-1} = Q_k^T$, dan $R_k = Q_k^T A_k$). Ini menunjukkan bahwa setiap iterasi adalah **transformasi similaritas**, sehingga nilai eigen tetap terjaga.

2.  **Konvergensi**:
    Iterasi dihentikan jika perubahan antara $A_{k+1}$ dan $A_k$ cukup kecil, yaitu:
    $$
    \|A_{k+1} - A_k\|_{\text{F}} < \text{toleransi}
    $$
    (Norma Frobenius sering digunakan di sini, atau norma lain yang sesuai).

3.  **Hasil**:
    * Jika $A$ simetris, $A_k$ akan konvergen ke **matriks diagonal**. Elemen-elemen diagonalnya adalah **nilai eigen** dari $A$.
    * Jika $A$ tidak simetris, $A_k$ akan konvergen ke **matriks segitiga atas** (atau bentuk Schur riil). Elemen-elemen diagonalnya adalah **nilai eigen** dari $A$. (Untuk nilai eigen kompleks, mungkin muncul blok 2x2 pada diagonal).

    Fungsi Anda mengembalikan `np.diag(A)` yang merupakan elemen-elemen diagonal dari matriks $A$ yang telah konvergen.

---

In [ ]:
import numpy as np

def QRmethod(A, max_iter=100, tol=1e-10):
    A = A.copy().astype(float)      # Membuat copy matriks agar menghindari error dan memastikan tipe data float
    for i in range(max_iter):
        Q, R = np.linalg.qr(A)      # Mendekomposisi QR (A = QR)
        A_next = R @ Q              # Menghitung A_next = R * Q
        # Mengecek konvergensi: jika perubahan antara A_next dan A cukup kecil
        if np.linalg.norm(A_next - A) < tol:
            break                   # Hentikan iterasi jika sudah konvergen
        A = A_next                  # Update A untuk iterasi berikutnya
    return np.diag(A)               # Mengembalikan diagonal dari matriks A yang telah konvergen


In [ ]:
# Matriks input
A = np.array([
    [1, 2, 3, 1, 5],
    [2, 3, 4, 5, 6],
    [3, 4, 5, 3, 7],
    [1, 5, 3, 4, 8],
    [5, 6, 7, 8, 9]
], dtype=float)
print("Matriks A:\n")
print(A)
print("Eigenvalues dari A:\n")
print(np.round(QRmethod(A), 5))

Matriks A:

[[1. 2. 3. 1. 5.]
 [2. 3. 4. 5. 6.]
 [3. 4. 5. 3. 7.]
 [1. 5. 3. 4. 8.]
 [5. 6. 7. 8. 9.]]
Eigenvalues dari A:

[24.24954 -3.57787  2.43565 -1.10732  0.     ]


**4. SVD(Singular Value Decomposition)**



Dekomposisi Nilai Singular (SVD) adalah faktorisasi fundamental dari sebuah matriks $A$ (bahkan matriks non-persegi) menjadi tiga matriks khusus. SVD memiliki aplikasi yang sangat luas dalam aljabar linear, analisis data, kompresi gambar, dan banyak bidang lainnya.

### Definisi dan Rumus Utama

Untuk setiap matriks $A$ berukuran $m \times n$, dekomposisi nilai singularnya diberikan oleh:

$$
A = U \Sigma V^T
$$

Dimana:
* $U$ adalah matriks ortogonal berukuran $m \times m$. Kolom-kolom $U$ disebut **vektor singular kiri**.
* $\Sigma$ (Sigma) adalah matriks diagonal $m \times n$ (dengan bentuk yang sama seperti $A$). Elemen-elemen diagonal non-negatif $\sigma_i$ dari $\Sigma$ disebut **nilai singular** (singular values) dari $A$. Nilai-nilai ini biasanya diurutkan dari yang terbesar hingga terkecil: $\sigma_1 \ge \sigma_2 \ge \dots \ge \sigma_r > 0$, dimana $r$ adalah rank dari $A$. Sisa elemen diagonal adalah nol.
  $$
  \Sigma = \text{diag}(\sigma_1, \sigma_2, \dots, \sigma_r, 0, \dots, 0)
  $$
* $V^T$ adalah transpos dari matriks ortogonal $V$ yang berukuran $n \times n$. Kolom-kolom $V$ (atau baris-baris $V^T$) disebut **vektor singular kanan**.


In [ ]:
def SVD(A):
    U, S, V = np.linalg.svd(A) #langsung memanggil fungsi SVD
    return S

In [ ]:
A = np.array([
    [1, 2, 3, 1, 5],
    [2, 3, 4, 5, 6],
    [3, 4, 5, 3, 7],
    [1, 5, 3, 4, 8]
], dtype=float)
print("Matrix A:\n")
print(A)
print("Eigenvalues of A:\n")
print(np.round(SVD(A),5))

Matrix A:

[[1. 2. 3. 1. 5.]
 [2. 3. 4. 5. 6.]
 [3. 4. 5. 3. 7.]
 [1. 5. 3. 4. 8.]]
Eigenvalues of A:

[18.46693  2.43849  2.28748  0.89102]
